## Description
The purpoes of this jupyter notebook is to investigate Random Forest train and predict boston housing price before implement in main .py (model_api.py).

* Model performance is evaluated using Mean Square Error (MSE) on validation set
* Use hyperparameter tuning to optimize the model's performance.

### Note: 
The Boston housing dataset is extracted from the URL: http://lib.stat.cmu.edu/datasets/boston

Since scikit-learn load_boston dataset is removed from version 1.2

### Outline Model Training Process:
1. Load Boston Housing Dataset
2. Preprocessing (optional)
3. Feature selection (optional)
4. Model Training
5. Hyperparameter Tuning

# 1. Loading Boston Housing Dataset 

In [4]:
import os
import pickle
import json
import pandas as pd
import numpy as np
import ssl
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [5]:
# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]) # 13 FEATURES
target = raw_df.values[1::2, 2] # MEDV

print(f"Boston dataset (features) shape: {data.shape}")
print(f"Boston dataset (target) shape: {target.shape}")

Boston dataset (features) shape: (506, 13)
Boston dataset (target) shape: (506,)


In [6]:
# Convert dataset to dataframe
columns = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", 
    "PTRATIO", "B", "LSTAT"
]

boston_df = pd.DataFrame(data, columns=columns)
boston_df['MEDV'] = target

# for the purpose of investigating the dataset
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# 3. Model Training

In [7]:
# validation and train split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=23)

In [8]:
param_grid = {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 3, 4]
    }

In [9]:
# Build model
rf_model = RandomForestRegressor()

# Grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtain best model
best_model = grid_search.best_estimator_
print(f"Best hyperparameters: {grid_search.best_params_}")

Best hyperparameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}


In [15]:
# Check available keys in cv_results_
available_keys = grid_search.cv_results_.keys()
available_keys

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_samples_leaf', 'param_min_samples_split', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [17]:
# Collecting artefacts and metrics for each trial
artefacts = []
for i, params in enumerate(grid_search.cv_results_['params']):
    test_score = -grid_search.cv_results_['mean_test_score'][i] 
    artefacts.append({
        'trial': i + 1,
        'hyperparameters': params,
        'test_score': test_score
    })

artefacts

[{'trial': 1,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'n_estimators': 100},
  'test_score': 10.154702455187023},
 {'trial': 2,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'n_estimators': 200},
  'test_score': 9.983101172738161},
 {'trial': 3,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'n_estimators': 300},
  'test_score': 9.806403149827013},
 {'trial': 4,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'n_estimators': 400},
  'test_score': 9.943093850947838},
 {'trial': 5,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 5,
   'n_estimators': 100},
  'test_score': 10.002419056164717},
 {'trial': 6,
  'hyperparameters': {'max_depth': None,
   'min_samples_leaf': 1,
   'min_samples_split': 5,
   'n_estimators': 200},
  'test_score': 9.67